In [ ]:
!pip install praw httpx pandas openai tqdm

In [ ]:
API_KEY = "INSERT YOUR KEY HERE"


In [ ]:
# pip install praw>=7.7
import os
import json
from datetime import datetime, timezone
import praw

# === Fill these OR set the env vars of the same names ===
CLIENT_ID = "INSERT YOUR CLIENT ID HERE"
CLIENT_SECRET = "INSERT YOUR CLIENT SECRET HERE"
USER_AGENT = "INSERT YOUR USER AGENT HERE"

# Put your Reddit post URL here:
POST_URL = "https://www.reddit.com/r/USCIS/comments/1m84yfm/2025_opt_timeline_continued"

# ---- create client (read-only) ----
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
    ratelimit_seconds=5,
)
reddit.read_only = True

# ---- load submission & sort comments by "new" ----
submission = reddit.submission(url=POST_URL)
submission.comment_sort = "new"

# Expand ALL "MoreComments" so nothing is missing
submission.comments.replace_more(limit=None)

# Flatten the tree
all_comments = submission.comments.list()

# Convert to simple JSON-serializable rows
rows = []
for c in all_comments:
    try:
        author = c.author.name if c.author else "[deleted]"
    except Exception:
        author = "[deleted]"
    rows.append({
        "post_id": submission.id,
        "post_title": submission.title,
        "post_permalink": f"https://www.reddit.com{submission.permalink}",
        "comment_id": c.id,
        "parent_id": c.parent_id,   # t1_... (comment) or t3_... (submission)
        "link_id": c.link_id,       # t3_<post_id>
        "author": author,
        "is_submitter": getattr(c, "is_submitter", False),
        "score": c.score,
        "body": c.body,
        "depth": c.depth,
        "created_utc": c.created_utc,
        "created_iso": datetime.fromtimestamp(c.created_utc, tz=timezone.utc).isoformat(),
        "permalink": f"https://www.reddit.com{c.permalink}",
    })

# Print everything as pretty JSON
print(json.dumps(rows, ensure_ascii=False, indent=2))


[
  {
    "post_id": "1m84yfm",
    "post_title": "2025 OPT Timeline continued",
    "post_permalink": "https://www.reddit.com/r/USCIS/comments/1m84yfm/2025_opt_timeline_continued/",
    "comment_id": "nc8odxb",
    "parent_id": "t3_1m84yfm",
    "link_id": "t3_1m84yfm",
    "author": "Content-Watch-3654",
    "is_submitter": false,
    "score": 1,
    "body": "I'm seeing stem opt approvals on April 23/24 but I don't see any update on my case (April 20 receipt date). What should I do?",
    "depth": 0,
    "created_utc": 1756925500.0,
    "created_iso": "2025-09-03T18:51:40+00:00",
    "permalink": "https://www.reddit.com/r/USCIS/comments/1m84yfm/2025_opt_timeline_continued/nc8odxb/"
  },
  {
    "post_id": "1m84yfm",
    "post_title": "2025 OPT Timeline continued",
    "post_permalink": "https://www.reddit.com/r/USCIS/comments/1m84yfm/2025_opt_timeline_continued/",
    "comment_id": "nc8973k",
    "parent_id": "t3_1m84yfm",
    "link_id": "t3_1m84yfm",
    "author": "ReasonableNature3

In [6]:
import pandas as pd
df = pd.DataFrame(rows)
df = df[df.depth==0]
df = df[["body"]]
df.to_csv("comments_raw.csv", index=False)

In [ ]:
# pip install openai tqdm
import os, json, time, random, threading
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
from openai import OpenAI
from httpx import HTTPError


client = OpenAI(api_key=API_KEY)


MODEL = "gpt-5-nano-2025-08-07"
OUTPUT_CSV = "comments_parsed.csv"
MAX_WORKERS = 16  # "nodes"

TARGET_COLS = [
    "application_type",
    "premium_processing",
    "receipt_date",
    "approved_date",
    "a_number_assigned",
    "card_produced_date",
    "card_shipped_date",
    "card_delivered_date",
    "premium_switch_date",   # NEW
    "notes",
]

# Ensure output columns exist
for col in TARGET_COLS:
    if col not in df.columns:
        df[col] = None

# Strict JSON schema for model output (no temperature used)
TEXT_FORMAT = {
    "format": {
        "type": "json_schema",
        "name": "opt_status",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "application_type": {"type": ["string", "null"]},
                "premium_processing": {
                    "anyOf": [
                        {"type": "string", "enum": ["Yes", "No"]},
                        {"type": "null"}
                    ]
                },
                "receipt_date": {
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "approved_date": {
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "a_number_assigned": {
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "card_produced_date": {
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "card_shipped_date": {
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "card_delivered_date": {
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "premium_switch_date": {  # NEW
                    "anyOf": [
                        {"type": "string", "pattern": r"^\d{4}-\d{2}-\d{2}$"},
                        {"type": "null"}
                    ]
                },
                "notes": {"type": ["string", "null"]}
            },
            "required": [
                "application_type",
                "premium_processing",
                "receipt_date",
                "approved_date",
                "a_number_assigned",
                "card_produced_date",
                "card_shipped_date",
                "card_delivered_date",
                "premium_switch_date",   # NEW
                "notes"
            ],
            "additionalProperties": False
        }
    }
}

SYSTEM_PROMPT = (
    "You are a precise parser. Extract fields from a Reddit comment about USCIS OPT/STEM OPT timelines.\n\n"
    "Return only the JSON fields required by the schema.\n"
    "Rules:\n"
    "- Accept label variants (e.g., 'Approved date' vs 'Approval Date').\n"
    "- Treat 'N/A', 'NA', 'Not yet', empty, or missing values as null.\n"
    "- Normalize 'Premium Processing' to 'Yes' or 'No' (else null).\n"
    "- Convert any dates to ISO 'YYYY-MM-DD'. Assume US MM/DD/YYYY unless the first number > 12 (then DD/MM/YYYY).\n"
    "- Extract 'premium_switch_date' when the user mentions switching to or filing Premium Processing; otherwise null.\n"
)

def call_model(comment_text: str, max_retries: int = 6) -> dict:
    """Call the Responses API with strict JSON schema. No temperature."""
    cleaned = (
        comment_text.replace("\u2060", "")
                    .replace("\ufeff", "")
                    .replace("\u200b", "")
                    .strip()
        or ""
    )
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": [
            {"type": "input_text", "text": f"Comment:\n<<<\n{cleaned}\n>>>"}  # treat the whole body as a block
        ]}
    ]

    delay = 1.0
    for attempt in range(max_retries):
        try:
            resp = client.responses.create(
                model=MODEL,
                input=messages,
                text=TEXT_FORMAT  # structured output
            )
            return json.loads(resp.output_text)
        except Exception as e:
            # backoff on transient errors or rate limits
            retryable = "RateLimit" in str(e) or "429" in str(e) or isinstance(e, HTTPError)
            if attempt < max_retries - 1 and retryable:
                time.sleep(delay + random.random())
                delay = min(delay * 2, 16)
                continue
            raise

def process_row(i: int, body: str) -> tuple[int, dict]:
    """Worker: parse one comment and return (index, result-dict)."""
    try:
        data = call_model(body)
    except Exception as err:
        data = {k: None for k in TARGET_COLS}
        data["notes"] = f"LLM error: {type(err).__name__}: {err}"
    return i, data

# Thread-safety for DataFrame update + CSV write
lock = threading.Lock()

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex, \
     tqdm(total=len(df), desc=f"Parsing with {MODEL} (x{MAX_WORKERS})") as pbar:

    futures = [ex.submit(process_row, i, body) for i, body in df["body"].items()]

    for fut in as_completed(futures):
        i, result = fut.result()

        with lock:
            for col in TARGET_COLS:
                df.at[i, col] = result.get(col, None)

            # Save progress after each completed row
            df.to_csv(OUTPUT_CSV, index=False)

        pbar.update(1)

print(f"Saved: {OUTPUT_CSV}")


Parsing with gpt-5-nano-2025-08-07 (x16):   0%|          | 0/220 [00:00<?, ?it/s]

Saved: comments_parsed.csv


In [11]:
import pandas as pd

IN = "merged.csv"
OUT = "merged.csv"

# The extracted date fields you’ve been using
date_cols = [
    "receipt_date",
    "premium_switch_date",
    "approved_date",
    "a_number_assigned",
    "card_produced_date",
    "card_shipped_date",
    "card_delivered_date",

]

# Keep only extracted columns (plus notes), then compute & sort by most recent date
keep_cols = [
    "application_type",
    "premium_processing",
    *date_cols,
]

df = pd.read_csv(IN)

# Keep only the columns you extracted (if any missing, they’ll be skipped)
keep_cols = [c for c in keep_cols if c in df.columns]
df = df[keep_cols].copy()

# Treat empty strings/whitespace as NA
df = df.replace(r"^\s*$", pd.NA, regex=True)

# Parse dates -> datetime64; invalid become NaT
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

# Row-wise most recent date across all date fields
df["most_recent_date"] = df[date_cols].max(axis=1)

# Sort by most recent date (descending). NaT (no dates) go last.
df = df.sort_values(by="most_recent_date", ascending=False, na_position="last").reset_index(drop=True)

# (Optional) convert datetimes to plain YYYY-MM-DD for a cleaner CSV
for c in date_cols + ["most_recent_date"]:
    df[c] = df[c].dt.date

df = df.drop(columns=["most_recent_date"])


df.to_csv(OUT, index=False)
print(f"Saved {OUT} with {len(df)} rows, sorted by most recent_date.")


Saved merged.csv with 638 rows, sorted by most recent_date.
